In [1]:
# Choose a theme to your convenience
PLOTLY_THEME = "plotly_dark"
#PLOTLY_THEME = "plotly_white"


In [2]:
from dataclasses import dataclass, asdict

@dataclass
class SimulationParameters:
    re: float
    N: int
    steps: int
    sampling: int
    K: float
    dt: float
    scheme: str

In [3]:
import configparser

params = SimulationParameters(100.0, 129, 50000, 5, 1.0, 1e-3, "IMEX")

config = configparser.ConfigParser()
config["Simulation"] = asdict(params)
with open('simulation.ini', 'w') as configfile:
  config.write(configfile)

In [4]:
import subprocess,  os

def launch_c():
    subprocess.run(["cmake", "--build", "build"])
    
    with open(os.devnull, "w") as devnull:
        proc = subprocess.Popen(
            ["./build/cldc"],
            stdout=devnull,
            stderr=devnull
        )

In [6]:
import mmap
import struct
import time

from tqdm.notebook import tqdm

import numpy as np
import plot_run
import posix_ipc


FLAT_ARRAY_SIZE = params.N * params.N * 2
record_size = params.steps // params.sampling
output_array = np.zeros((record_size, params.N, params.N, 2))

shm = posix_ipc.SharedMemory("/sim_shm", posix_ipc.O_CREAT, size=8 + 8 * FLAT_ARRAY_SIZE)
mm = mmap.mmap(shm.fd, shm.size)

shm.close_fd()

mm.write(struct.pack("Q", 0))

write_idx = np.frombuffer(mm, dtype=np.uint64, count=1, offset=0)
data = np.frombuffer(mm, dtype=np.float64, count=FLAT_ARRAY_SIZE, offset=8)

plotter = plot_run.RunPlot(params.re, params.K, params.N, PLOTLY_THEME)
plotter.prepare()
last = 0
try:
    pbar = tqdm(total=params.steps)
    pbar.set_description("Waiting for C simulation")
    launch_c()
    while True:
        w = int(write_idx[0])
        index = w // params.sampling
        if w > last:
            pbar.set_description("")
            pbar.update(params.sampling)
            last = w
            frame = data.copy().reshape(params.N, params.N, 2)

            output_array[index] = frame
            plotter.update(frame, index)
        if index >= record_size - 1:
            break
        time.sleep(0.001)
except KeyboardInterrupt:
    pass
print("Saving run file")
np.save("runs/out", output_array)


FigureWidget({
    'data': [{'line': {'width': 2},
              'mode': 'lines',
              'name': 'min',
              'type': 'scatter',
              'uid': 'df9eed24-0543-45a8-88c2-71c36d81e775',
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'width': 2},
              'mode': 'lines',
              'name': 'mean',
              'type': 'scatter',
              'uid': '09e4afe8-bbf9-4e41-8604-2f25a5eafa74',
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'width': 2},
              'mode': 'lines',
              'name': 'max',
              'type': 'scatter',
              'uid': 'feefb6a4-68f6-49ff-9976-ed4836a5aded',
              'xaxis': 'x',
              'y': [],
              'yaxis': 'y'},
             {'line': {'width': 2},
              'mode': 'lines',
              'name': 'profile',
              'type': 'scatter',
              'uid': 'c01078d0-7d64-4f5e-

  0%|          | 0/50000 [00:00<?, ?it/s]

[100%] Built target cldc


/home/kz/vault/all-cfd/CFD-Solver/.venv/lib/python3.13/site-packages/jupyter_client/session.py:722: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



Saving run file
